In [ ]:
import requests
import os
from tqdm import tqdm
tqdm.pandas()

import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt


nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt_tab")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shaun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shaun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Shaun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
def download_file(url, filename):
    if os.path.exists(filename):
        print(f"File '{filename}' already exists. Skipping download.")
        return

    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            total_size_in_bytes = int(r.headers.get("content-length", 0))
            block_size = 16384
            progress_bar = tqdm(total=total_size_in_bytes, unit="iB", unit_scale=True)
            with open(filename, "wb") as file:
                for data in r.iter_content(block_size):
                    progress_bar.update(len(data))
                    file.write(data)
            progress_bar.close()
            if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
                print("ERROR, something went wrong")
            else:
                print(f"File downloades successfully as {filename}")

    except requests.exceptions.RequestException as e:
        print(f"Error during download: {e}")

file_url = "https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/Appliances.jsonl.gz"
local_filename = "Appliances.jsonl.gz"
download_file(file_url, local_filename)

File 'Appliances.jsonl.gz' already exists. Skipping download.


In [3]:
df = pd.read_json("Appliances.jsonl.gz", compression="gzip", lines=True)

In [34]:
reviews_df = df[["text"]].copy()

In [35]:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

def tokenize(text):
    return nltk.word_tokenize(text)

stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

lemmatizer = WordNetLemmatizer()
def lemmatize(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

reviews_df['text'] = reviews_df['text'].progress_apply(clean_text)
reviews_df['tokens'] = reviews_df['text'].progress_apply(tokenize)
reviews_df['tokens'] = reviews_df['tokens'].progress_apply(remove_stopwords)
reviews_df['tokens'] = reviews_df['tokens'].progress_apply(lemmatize)

100%|██████████| 100000/100000 [00:08<00:00, 11947.27it/s]


In [36]:
dictionary = corpora.Dictionary(reviews_df['tokens'])
corpus = [dictionary.doc2bow(tokens) for tokens in reviews_df['tokens']]

In [37]:
num_topics = 5
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

In [38]:
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')


Topic: 0 
Words: 0.019*"one" + 0.018*"part" + 0.011*"old" + 0.011*"new" + 0.010*"dryer" + 0.008*"year" + 0.008*"would" + 0.007*"month" + 0.007*"working" + 0.007*"back"

Topic: 1 
Words: 0.044*"ice" + 0.041*"coffee" + 0.026*"use" + 0.020*"make" + 0.019*"cup" + 0.016*"love" + 0.016*"maker" + 0.013*"great" + 0.012*"work" + 0.010*"easy"

Topic: 2 
Words: 0.010*"br" + 0.008*"use" + 0.008*"machine" + 0.007*"get" + 0.007*"like" + 0.007*"little" + 0.007*"well" + 0.007*"would" + 0.006*"washer" + 0.006*"stove"

Topic: 3 
Words: 0.082*"great" + 0.072*"fit" + 0.058*"work" + 0.052*"easy" + 0.036*"perfect" + 0.036*"install" + 0.035*"good" + 0.033*"product" + 0.032*"perfectly" + 0.027*"price"

Topic: 4 
Words: 0.067*"filter" + 0.050*"water" + 0.017*"one" + 0.012*"brand" + 0.011*"good" + 0.011*"much" + 0.011*"better" + 0.011*"taste" + 0.011*"work" + 0.011*"price"



In [39]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.031915  0.192378       1        1  29.282509
2      0.145422 -0.083108       2        1  26.985762
4      0.050038  0.147107       3        1  16.271703
1      0.129790 -0.190950       4        1  15.724700
3     -0.357166 -0.065427       5        1  11.735326, topic_info=         Term          Freq         Total Category  logprob  loglift
23      great  20145.000000  20145.000000  Default  30.0000  30.0000
36        fit  16710.000000  16710.000000  Default  29.0000  29.0000
12     filter  20375.000000  20375.000000  Default  28.0000  28.0000
17      water  13774.000000  13774.000000  Default  27.0000  27.0000
293       ice  11304.000000  11304.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
91       like   2167.483897  11177.820806   Topic5  -4.4806   0.5022
75   original   1552.581232   4915.683772   Topic5  -4.8143   0.9901
46       part   1844.510344  10337.198707   Topic5  -4.6420   0.4190
416      item   1157.592265   3800.590438   Topic5  -5.1078   0.9538
434      look   1108.875332   4521.634691   Topic5  -5.1508   0.7370

[342 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
195       4  0.045148  advertised
195       5  0.954957  advertised
9         1  0.175306        also
9         2  0.519587        also
9         3  0.106401        also
...     ...       ...         ...
86        5  0.045202       would
87        1  0.669658        year
87        3  0.330299        year
62        1  0.999037     youtube
2974      3  0.996835        zero

[511 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 5, 2, 4])

In [40]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3439691662788391	 
Topic: 0.010*"br" + 0.008*"use" + 0.008*"machine" + 0.007*"get" + 0.007*"like" + 0.007*"little" + 0.007*"well" + 0.007*"would" + 0.006*"washer" + 0.006*"stove"

Score: 0.29765933752059937	 
Topic: 0.044*"ice" + 0.041*"coffee" + 0.026*"use" + 0.020*"make" + 0.019*"cup" + 0.016*"love" + 0.016*"maker" + 0.013*"great" + 0.012*"work" + 0.010*"easy"

Score: 0.1925359070301056	 
Topic: 0.082*"great" + 0.072*"fit" + 0.058*"work" + 0.052*"easy" + 0.036*"perfect" + 0.036*"install" + 0.035*"good" + 0.033*"product" + 0.032*"perfectly" + 0.027*"price"

Score: 0.1455492377281189	 
Topic: 0.067*"filter" + 0.050*"water" + 0.017*"one" + 0.012*"brand" + 0.011*"good" + 0.011*"much" + 0.011*"better" + 0.011*"taste" + 0.011*"work" + 0.011*"price"

Score: 0.020286384969949722	 
Topic: 0.019*"one" + 0.018*"part" + 0.011*"old" + 0.011*"new" + 0.010*"dryer" + 0.008*"year" + 0.008*"would" + 0.007*"month" + 0.007*"working" + 0.007*"back"


In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_df['tokens'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.367801829609589

Coherence Score:  0.5366377020780211


In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = models.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=15)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

start = 5
limit = 30
step = 5
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=reviews_df['tokens'], start=start, limit=limit, step=step)

x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))